In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')  # read df from csv file

df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H;%M:%S')
df = df.drop(columns=['Date', 'Time'])
df.set_index('Datetime', inplace=True)

# using month and hour as features
df['Month'], df['Day'], df['Hour']  = df.index.month, df.index.day, df.index.hour 

print(df[:5])

In [ ]:
target_date=['2007-11-09', '2007-11-08']

def plot_data(df, target_date):
    for tar in target_date:
        one_day_data = df.loc[tar]
        ax = one_day_data.ewm(alpha=1/50, adjust=True).mean().plot(y='Load', figsize=(10, 3))
        one_day_data.plot(y='Load', figsize=(10, 3), ax=ax)

plot_data(df, target_date)

In [ ]:
import numpy as np
import torch
from demand_utils import *

device="cuda"

# preprocessing
preprocess_info = {}
for col in ['Month', 'Day', 'Hour']:
    preprocess_info[col] = (df[col].mean(), df[col].std())
    df[col] = (df[col] - df[col].mean()) / df[col].std()
    
print(preprocess_info)

n_train = int(len(df) * 0.8)
train_features = torch.tensor(df.iloc[:n_train, 1:].values, dtype=torch.float32, device=device)
test_features = torch.tensor(df.iloc[n_train:, 1:].values, dtype=torch.float32, device=device)
train_labels = torch.tensor(df.iloc[:n_train, 0].values.reshape(-1, 1), dtype=torch.float32, device=device)
test_labels = torch.tensor(df.iloc[n_train:, 0].values.reshape(-1, 1), dtype=torch.float32, device=device)
print(train_features.shape, train_labels.shape)

In [ ]:
demand_dataset = Demand_Dataset(train_features, train_labels)
demand_dataloader = DataLoader(demand_dataset, batch_size=36, shuffle=True)
demand_model = Demand_Model(device)


PATH = './demand_model_weights.pth'
demand_model.load_state_dict(torch.load(PATH))

# # training
# num_epochs = 100
# for epoch in range(num_epochs):
#     for batch_idx, (features, labels) in enumerate(demand_dataloader):
#         loss = demand_model.update(features, labels)
#     print(loss)

# torch.save(demand_model.state_dict(), 'demand_model_weights.pth')

In [ ]:
# testing
import matplotlib.pyplot as plt
pred_vals = []
real_vals = []

start = 2000
steps = 10000

for i in range(start, start + steps):
    pred = demand_model.transition_step(train_features[i])
    pred_vals.append(pred.item())
    real_vals.append(train_labels[i].item())

fig, (ax1, ax2) = plt.subplots(2, 1)
fig.set_figheight(30)
fig.set_figwidth(50)
ax1.plot(np.arange(0, steps), pred_vals, label='Predicted')
ax2.plot(np.arange(0, steps), real_vals, label='Actual')